In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy import stats

germanCredit = pd.read_csv("germancredit.csv")
germanCredit['installment'] = germanCredit['installment'].astype(object)
germanCredit['residence'] = germanCredit['residence'].astype(object)
germanCredit['cards'] = germanCredit['cards'].astype(object)
germanCredit['liable'] = germanCredit['liable'].astype(object)

############# a)
# Separating qualitative and quantitative predictors
qual = []
quan = []

for col in germanCredit.columns:
    if germanCredit[col].dtype == 'object':  # Assuming object dtype represents factors
        qual.append(col)
    else:
        quan.append(col)

#remove default
quan = quan[1:]

data_qual = germanCredit[qual]
data_quan = germanCredit[quan]

# Your data and col/qual definitions here...

num_qual_cols = len(qual)
num_quan_cols = len(quan)

# Determine the number of rows for barplots (4 columns per row)
num_barplot_rows = (num_qual_cols + 3) // 4

# Create a 4x3 grid for barplots
plt.figure(figsize=(15, 5 * num_barplot_rows))
for i, col in enumerate(qual):
    if col != 'Default':
        plt.subplot(num_barplot_rows, 4, i + 1)
        sns.countplot(x=col, hue="Default", data=germanCredit)
        plt.xlabel("Default Status")
        plt.ylabel(col)
        plt.title(f"Barplot for {col}")

# Create a 1x3 grid for boxplots
plt.figure(figsize=(15, 5))
for i, col in enumerate(quan):
    plt.subplot(1, num_quan_cols, i + 1)
    sns.boxplot(x="Default", y=col, data=germanCredit)
    plt.xlabel("Default status")
    plt.ylabel(col)
    plt.title(f"Boxplot for {col}")

plt.tight_layout()  # Ensure proper spacing between subplots
plt.show()

#### significance of quantitatve predictor variables

# Create an empty list to store p-values
p_values = []

# Iterate through the predictor variables
for col in quan:
    formula = f'Default ~ {col}'
    model = smf.glm(formula=formula, data=germanCredit, family=sm.families.Binomial())
    fit = model.fit()

    # Extract the p-value for the variable
    p_value = fit.pvalues[col]

    # Print the p-value for the variable
    print(f'P-Value for {col}: {p_value}')

    # Append the p-value to the list
    p_values.append(p_value)

#### significance of qualitative predictor variables
formula_intercept = 'Default ~ 1'
model_intercept = smf.glm(formula=formula_intercept, data=germanCredit, family=sm.families.Binomial())
fit_intercept = model_intercept.fit()

for col in qual[1:]:
  formula = f'Default ~ {col}'
  model = smf.glm(formula=formula, data=germanCredit, family=sm.families.Binomial())
  fit = model.fit()

  lr_test = 2 * (fit.llf - fit_intercept.llf)
  df = fit.df_model - fit_intercept.df_model
  p_value = 1 - stats.chi2.cdf(lr_test, df)

  # Print the LRT results
  print(f'Summary for {col}:')
  print(f"LRT Statistic: {lr_test}")
  print(f"Degrees of Freedom: {df}")
  print(f"P-Value: {p_value}")
  print("\n")

############# b)

# Initial formula with all variables
formula = 'Default ~ checkingstatus1 + duration + history + purpose + amount + savings + employ + installment + status + others + residence + property + age + otherplans + housing + cards + job + liable + tele + foreign'
model = smf.glm(formula=formula, data=germanCredit, family=sm.families.Binomial())
fit = model.fit()

# Create a list of variables to be removed
variables_to_remove = ['employ', 'property','age', 'cards', 'job', 'liable']
for variable in variables_to_remove:
    # Remove the current variable from the formula
    formula = formula.replace(f" + {variable}", "")
    formula = formula.replace(f" {variable} + ", " + ")

    # Create a new model and fit it
    model = smf.glm(formula=formula, data=germanCredit, family=sm.families.Binomial())
    new_fit = model.fit()

    # Perform likelihood ratio test (LRT)
    lr_test = 2 * (fit.llf - new_fit.llf)
    df = fit.df_model - new_fit.df_model
    p_value = 1 - stats.chi2.cdf(lr_test, df)

    # Print the LRT results
    print(f"Removing variable {variable}:")
    print(f"LRT Statistic: {lr_test}")
    print(f"Degrees of Freedom: {df}")
    print(f"P-Value: {p_value}")
    print("\n")
    print(f"AIC: {new_fit.aic}")

    # Update fit to the new_fit for the next iteration
    fit = new_fit

############# c)

lr_prob = new_fit.predict(germanCredit)

# Predict binary outcomes based on a threshold (0.5 in this case)
lr_pred = (lr_prob >= 0.5).astype(int)

# Calculate training error rate
training_error_rate = 1 - (lr_pred == germanCredit['Default']).mean()
print(f"Training Error Rate: {training_error_rate:.3f}")


############################################################

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy import stats

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import KFold, LeaveOneOut, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

germanCredit = pd.read_csv("germancredit.csv")
germanCredit['installment'] = germanCredit['installment'].astype(object)
germanCredit['residence'] = germanCredit['residence'].astype(object)
germanCredit['cards'] = germanCredit['cards'].astype(object)
germanCredit['liable'] = germanCredit['liable'].astype(object)

X = germanCredit.loc[:, germanCredit.columns != 'Default']
y = germanCredit['Default']

####### part a)

formula = 'Default ~ checkingstatus1 + duration + history + purpose + amount + savings + employ + installment + status + others + residence + property + age + otherplans + housing + cards + job + liable + tele + foreign'
model = smf.glm(formula=formula, data=germanCredit, family=sm.families.Binomial())
fit1 = model.fit()
print(fit1.summary())

lr_prob = fit1.predict(X)
lr_pred = np.empty(lr_prob.shape, dtype=object)
lr_pred[lr_prob<0.5] = 0
lr_pred[lr_prob>=0.5] = 1

## Training error rate
np.mean(lr_pred != germanCredit['Default'])
np.mean(lr_pred != y)

confusion_matrix = pd.crosstab(germanCredit['Default'], lr_pred, rownames=['Actual'], colnames=['Predicted'])
print(confusion_matrix)

# Calculate sensitivity (True Positive Rate)
TP = confusion_matrix.iloc[1, 1]  # True Positives
FN = confusion_matrix.iloc[1, 0]  # False Negatives
sensitivity = TP / (TP + FN)

# Calculate specificity
TN = confusion_matrix.iloc[0, 0]  # True Negatives
FP = confusion_matrix.iloc[0, 1]  # False Positives
specificity = TN / (TN + FP)

print(f"Sensitivity (True Positive Rate): {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")

###### part b)

misClass_Error = np.zeros(len(germanCredit))
for i in range(len(germanCredit)):
    #dataTest = germanCredit.loc[i, :]
    dataTest = germanCredit.loc[i:i, :].copy()
    dataTrain = germanCredit.drop(i, axis=0)

    formula = 'Default ~ checkingstatus1 + duration + history + purpose + amount + savings + employ + installment + status + others + residence + property + age + otherplans + housing + cards + job + liable + tele + foreign'
    model = smf.glm(formula = formula, data = dataTrain, family=sm.families.Binomial())
    fit = model.fit()

    lr_prob = fit.predict(dataTest)
    lr_pred = np.empty(lr_prob.shape, dtype=object)
    lr_pred[lr_prob<0.5] = 0
    lr_pred[lr_prob>=0.5] = 1

    misClass_Error[i] = np.mean(lr_pred != dataTest['Default'])

print('Training error: = (%.3f)' % np.mean(misClass_Error))

###### part c)

preds, scores = [], []
kfold = KFold(n_splits=germanCredit.shape[0])

for train_idx, test_idx in kfold.split(germanCredit):
    X_train, X_test = X.iloc[train_idx,:], X.iloc[test_idx,:]
    y_test = y[test_idx]
    formula = 'Default ~ checkingstatus1 + duration + history + purpose + amount + savings + employ + installment + status + others + residence + property + age + otherplans + housing + cards + job + liable + tele + foreign'
    model = smf.glm(formula = formula, data = germanCredit.iloc[train_idx,:], family=sm.families.Binomial()).fit()
    scores.append(model.predict(X_test))
    preds.append(1* (model.predict(X_test)>=0.5) == y_test)

print('Training error: = (%.4f)' % (1- np.mean(np.array(preds))))

######### part d)

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()
# Define the list of categorical columns to be encoded
categorical_columns = ['checkingstatus1', 'history', 'purpose', 'savings', 'employ', 'installment',
                       'status', 'others','residence', 'property', 'otherplans','housing','cards', 'job','liable', 'tele', 'foreign']
# Apply label encoding to each categorical column
for column in categorical_columns:
  germanCredit[column] = label_encoder.fit_transform(germanCredit[column])

X = germanCredit.loc[:, germanCredit.columns != 'Default']
y = germanCredit['Default']

loo = LeaveOneOut()
lda = LinearDiscriminantAnalysis()

lda.fit(X, y)

ldapred_train = lda.predict(X)

misclassification_train = 1 - accuracy_score(y, ldapred_train)
print(f"\nTraining Misclassification Rate: {misclassification_train:.3f}")

conf_mat_train = pd.crosstab(ldapred_train, y, rownames = ['predict'], colnames = ['train'])
print("\n Confusion Matrix for train data \n", conf_mat_train)

# Calculate sensitivity (True Positive Rate)
TP = conf_mat_train.iloc[1, 1]  # True Positives
FN = conf_mat_train.iloc[1, 0]  # False Negatives
sensitivity = TP / (TP + FN)

# Calculate specificity
TN = conf_mat_train.iloc[0, 0]  # True Negatives
FP = conf_mat_train.iloc[0, 1]  # False Positives
specificity = TN / (TN + FP)

print(f"Sensitivity (True Positive Rate): {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")

#calculate Training error using LOOCV
lda_score = cross_val_score(estimator=lda, X=X, y=y, cv=loo)
print('Training error: = (%.3f)' % (1- np.mean(lda_score)))

probs = lda.predict_proba(X)
probs_one = [i[1] for i in probs]
# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y, probs_one)
sns.lineplot(x = fpr, y = tpr).set(xlabel = "FPR", ylabel = "TPR", title = "ROC Curve LDA")


####### part e)

loo = LeaveOneOut()
qda = QuadraticDiscriminantAnalysis()

qda.fit(X, y)

qdapred_train = qda.predict(X)

misclassification_train = 1 - accuracy_score(y, qdapred_train)
print(f"\nTraining Misclassification Rate: {misclassification_train:.3f}")

conf_mat_train = pd.crosstab(qdapred_train, y, rownames = ['predict'], colnames = ['train'])
print("\n Confusion Matrix for train data \n", conf_mat_train)

# Calculate sensitivity (True Positive Rate)
TP = conf_mat_train.iloc[1, 1]  # True Positives
FN = conf_mat_train.iloc[1, 0]  # False Negatives
sensitivity = TP / (TP + FN)

# Calculate specificity
TN = conf_mat_train.iloc[0, 0]  # True Negatives
FP = conf_mat_train.iloc[0, 1]  # False Positives
specificity = TN / (TN + FP)

print(f"Sensitivity (True Positive Rate): {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")

#calculate Training error using LOOCV
qda_score = cross_val_score(estimator=qda, X=X, y=y, cv=loo)
print('Training error: = (%.3f)' % (1- np.mean(qda_score)))

probs = qda.predict_proba(X)
probs_one = [i[1] for i in probs]
# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y, probs_one)
sns.lineplot(x = fpr, y = tpr).set(xlabel = "FPR", ylabel = "TPR", title = "ROC Curve QDA")

######### parts f)

knn2 = KNeighborsClassifier()
param_grid = {'n_neighbors': np.arange(1, 10)}
knn_gscv = GridSearchCV(knn2, param_grid, cv=loo)
knn_gscv.fit(X, y)
print('K that minimize test error: ', knn_gscv.best_params_)

knn = KNeighborsClassifier(n_neighbors=77)
knn.fit(X, y)

knnpred_train = knn.predict(X)

misclassification_train = 1 - accuracy_score(y, knnpred_train)
print(f"\nTraining Misclassification Rate: {misclassification_train:.3f}")

conf_mat_train = pd.crosstab(knnpred_train, y, rownames = ['predict'], colnames = ['train'])
print("\n Confusion Matrix for train data \n", conf_mat_train)

# Calculate sensitivity (True Positive Rate)
TP = conf_mat_train.iloc[1, 1]  # True Positives
FN = conf_mat_train.iloc[1, 0]  # False Negatives
sensitivity = TP / (TP + FN)

# Calculate specificity
TN = conf_mat_train.iloc[0, 0]  # True Negatives
FP = conf_mat_train.iloc[0, 1]  # False Positives
specificity = TN / (TN + FP)

print(f"Sensitivity (True Positive Rate): {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")

#calculate training error using LOOCV
print('\n Training error: = (%.4f)' % (1 - knn_gscv.best_score_))

probs = knn.predict_proba(X)
probs_one = [i[1] for i in probs]
# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y, probs_one)
sns.lineplot(x = fpr, y = tpr).set(xlabel = "FPR", ylabel = "TPR", title = "ROC Curve KNN")

######### parts g)

germanCredit = pd.read_csv("germancredit.csv")
germanCredit['installment'] = germanCredit['installment'].astype(object)
germanCredit['residence'] = germanCredit['residence'].astype(object)
germanCredit['cards'] = germanCredit['cards'].astype(object)
germanCredit['liable'] = germanCredit['liable'].astype(object)

X = germanCredit.loc[:, germanCredit.columns != 'Default']
y = germanCredit['Default']

formula = 'Default ~ checkingstatus1 + duration + history + purpose + amount + savings + installment + status + others + residence + otherplans + housing + tele + foreign'
model = smf.glm(formula=formula, data=germanCredit, family=sm.families.Binomial())
red_fit = model.fit()

lr_prob = red_fit.predict(X)
lr_pred = np.empty(lr_prob.shape, dtype=object)
lr_pred[lr_prob<0.5] = 0
lr_pred[lr_prob>=0.5] = 1

## Training error rate
np.mean(lr_pred != germanCredit['Default'])
np.mean(lr_pred != y)

confusion_matrix = pd.crosstab(germanCredit['Default'], lr_pred, rownames=['Actual'], colnames=['Predicted'])
print(confusion_matrix)

# Calculate sensitivity (True Positive Rate)
TP = confusion_matrix.iloc[1, 1]  # True Positives
FN = confusion_matrix.iloc[1, 0]  # False Negatives
sensitivity = TP / (TP + FN)

# Calculate specificity
TN = confusion_matrix.iloc[0, 0]  # True Negatives
FP = confusion_matrix.iloc[0, 1]  # False Positives
specificity = TN / (TN + FP)

print(f"Sensitivity (True Positive Rate): {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")

misClass_Error = np.zeros(len(germanCredit))
for i in range(len(germanCredit)):
    #dataTest = germanCredit.loc[i, :]
    dataTest = germanCredit.loc[i:i, :].copy()
    dataTrain = germanCredit.drop(i, axis=0)

    formula = 'Default ~ checkingstatus1 + duration + history + purpose + amount + savings + installment + status + others + residence + otherplans + housing + tele + foreign'
    model = smf.glm(formula = formula, data = dataTrain, family=sm.families.Binomial())
    fit = model.fit()

    lr_prob = fit.predict(dataTest)
    lr_pred = np.empty(lr_prob.shape, dtype=object)
    lr_pred[lr_prob<0.5] = 0
    lr_pred[lr_prob>=0.5] = 1

    misClass_Error[i] = np.mean(lr_pred != dataTest['Default'])

######################################################

import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy import stats

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, LeaveOneOut, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score

germanCredit = pd.read_csv("germancredit.csv")
cat_preds = ["checkingstatus1", "history", "purpose", "savings", "employ",
"status", "others", "property", "otherplans", "housing",
"job", "tele", "foreign", "liable"]
# Encode categorical predictors
germanCredit = pd.get_dummies(data = germanCredit, columns = cat_preds, drop_first = True)

X = germanCredit.loc[:, germanCredit.columns != 'Default']
y = germanCredit['Default']

### a) logistic regression model using all predictors

# Standardize features (important for logistic regression)
X_scaled = X.values

# Create a logistic regression model
logistic_model = LogisticRegression(solver='liblinear')

# Create Leave-One-Out cross-validation
loocv = LeaveOneOut()

# Initialize variables to store accuracy scores
accuracy_scores = []

# Perform LOOCV
for train_index, test_index in loocv.split(X_scaled):
X_train, X_test = X_scaled[train_index], X_scaled[test_index]
y_train, y_test = y[train_index], y[test_index]

# Fit the model on the training data
logistic_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = logistic_model.predict(X_test)

# Calculate and store accuracy
accuracy = accuracy_score(y_test, y_pred)
accuracy_scores.append(accuracy)

# Calculate the mean accuracy over all LOOCV iterations
mean_accuracy = np.mean(accuracy_scores)

print("Mean Accuracy using LOOCV:", mean_accuracy)


### (b), (c) Fit Ridge and Lasso logistic regression model

# Create a logistic regression model
logistic_model = LogisticRegression(solver='liblinear')

# Define a grid of hyperparameters to search
param_grid_l1 = {
	'penalty': ['l1'],  # L1 (Lasso) regularization
	'C': np.logspace(-4, 1, 5)  # A range of regularization strengths
}

param_grid_l2 = {
	'penalty': ['l2'],  # L2 (Ridge) regularization
	'C': np.logspace(-4, 4, 10)  # A range of regularization strengths
}

# Create Leave-One-Out cross-validation
loo = LeaveOneOut()

# Create GridSearchCV objects for L1 and L2
grid_search_l1 = GridSearchCV(logistic_model, param_grid_l1, cv=loo, scoring='accuracy')
grid_search_l2 = GridSearchCV(logistic_model, param_grid_l2, cv=loo, scoring='accuracy')

# Fit the models to the data to perform grid search
grid_search_l1.fit(X, y)
grid_search_l2.fit(X, y)

# Get the best parameters and best scores for L1 and L2
best_params_l1 = grid_search_l1.best_params_
best_score_l1 = grid_search_l1.best_score_

best_params_l2 = grid_search_l2.best_params_
best_score_l2 = grid_search_l2.best_score_

print("Best Parameters for L1:", best_params_l1)
print("Best Score for L1:", best_score_l1)

print("Best Parameters for L2:", best_params_l2)
print("Best Score for L2:", best_score_l2)